# Hyperparameter Tuning using HyperDrive


In [1]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.core.conda_dependencies import CondaDependencies
#from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.estimator import Estimator

In [2]:
# Creation of compute cluster to carry our the automated ML
ws = Workspace.from_config()
compute_name = "udacity-cluster"
try:
    compute = ComputeTarget(workspace=ws, name=compute_name)
    print('Compute cluster {} already exists!'.format(compute_name))
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute = ComputeTarget.create(ws, compute_name, config)
    
compute.wait_for_completion()

## Dataset

In [3]:
# TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.
dataset_name = 'color_shades'
if dataset_name in ws.datasets.keys():
        dataset = ws.datasets[dataset_name] 
else:
        url = "https://raw.githubusercontent.com/zgoey/azure_ml_capstone/master/color_shades.csv"
        dataset = Dataset.Tabular.from_delimited_files(url)        
        dataset.register(workspace = ws, name = dataset_name,
                                 description = 'RGB values labeled with color shade names',
                                 create_new_version = True)

In [4]:
ws = Workspace.from_config()
experiment_name = 'udacity-capstone'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

For this hyperdrive experiment we will use a K-nearest-neighbor classifier (kNN), which is simple and at the same time flexible enough to adapt itself to irregular decision boundaries. For a K-nearest neighvor classifier it is always a bit of challenge to find the optimal number of neighbors, so that is one of the parameters, over which we will sample. We will also investigate the effect of weighting samples by their distances as opposed to giving all neighbors the same weight by including the 'weights' parameter in our search . Finally, the distance measure is something that is interesting to vary. Here we will take an educated guess. Instead of varyting the metric, we will compare a direct kNN application to one preceded by an embedding into the L*a*b* space that was designed to be perceprually uniform (see https://en.wikipedia.org/wiki/CIELAB_color_space) and a metric learning algporithm that Neighborhood Components Analysis (see https://scikit-learn.org/stable/modules/neighbors.html, section 1.6.7), which has been devised as an improvement kNN with the standard Euclidean distance metric.

We use Bayesian parameter sampling, because our hyperparamter sample space is relatively small and we have enough budget to explore. We do no set an early termination policy, because this is not supported when using Bayesian sampling. The maximal number of runs is set to 100, which is according to the recommendation to the recoomendation to set the maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned (see https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#define-search-space). The maximum number of concurrent runs is set to 1, to let each run beenfit fullt from previously completed runs, which will enhance the sampling convergence.

We set the primary metric to accuracy, which is the most common measure used for classification tasks. Further details of the traning procesure can be found in the file train.py, which defined the estimator that we use in our experiment.


In [13]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.

#TODO: Create the different params that you will be using during training
param_sampling =   BayesianParameterSampling(
    {
        '--n_neighbors': choice(range(1,101)),
        '--weights': choice(range(2)),
        '--embedding': choice(range(3))
    }
)

#TODO: Create your estimator and hyperdrive config
# estimator = Estimator(source_directory=".",
#                 entry_script="train.py",
#                 compute_target=compute,
#                 conda_packages=['scikit-learn==0.21.3', 'pandas==0.23.4']
#                )

# hyperdrive_run_config = HyperDriveConfig(
#     estimator=estimator,
#     hyperparameter_sampling=param_sampling,
#     primary_metric_name='Accuracy',
#     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
#     max_total_runs = 150,
#     max_concurrent_runs=1
#     )

#  We got a warning saying:
#  WARNING:azureml.train.estimator._estimator:'Estimator' is deprecated. Please use 'ScriptRunConfig' from 
# 'azureml.core.script_run_config' with your own defined environment or an Azure ML curated environment. 
#  Because of this, we are now using ScriptRunConfig instead.

env = Environment("hyperparameter_environment")
env.docker.enabled = True
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn',
                                                                         'scikit-image',
                                                                         'pandas',
                                                                         'numpy'
                                                                         ])
env.python.conda_dependencies.save_to_file(".", "hyperparameter_dependencies.yml")

src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=compute,
                      environment=env)
hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=1)

In [14]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?


In [15]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion()

## Best Model

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())
print(best_run.get_file_names())

In [ ]:
#TODO: Save the best model
os.makedirs('./models', exist_ok=True)
best_run.download_file(name='outputs/model.pkl', output_file_path='./models/hyperdrive_color_shades.pkl')
best_run.register_model(model_name='hyperdrive_color_shades', model_path='outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service